 The code in this notebook, **when run in order**, will:

#### (1) Produce a dataframe of tweets for a certain keyword (columns right now are, location and tweet text; can modify the search criteria as necessary (location lists, hashtags, etc.), saved as a csv

#### (2) Delete the duplicates/very similar spam tweets but leave one copy behind. 

#### (3) Clean the remaining tweets and saves as a csv

By the end of the notebook, you'll have a clean twitter dataframe that's ready for analysis 

In [1]:
import os
import tweepy
import pandas as pd
import time
import itertools
import numpy as np
from itertools import chain


In [2]:
consumer_key= '3emnsuDpqLBXlxD5UPUxVnxTt'
consumer_secret= 'ZPZBilbjbca7pYAmMv05hlNKsxLb4CJd6A5kNa2JNIs62BjF7m'
access_token= '1406213154-M77vYKXz7wIAUzzJkqLVzfDH0D6K11xMJkzGHzJ'
access_token_secret= 'iLcP4BuW4jWRYIHEiIZwsD5jgYlMY1aTmtSFwPQqbaQrX'

In [3]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

## Specify tweet search/filtering criteria here:

In [4]:
search_words = "mask free"
date_since = "2021-03-10"
new_search = search_words + " -filter:retweets"


## Set location labels and corresponding location coordinates and radii here:

In [26]:
## ** In the order of the counties listed in the google doc, and put in the Facebook chat

locs_labels = ['Texas test',
'New York County NY',
 'County of San Francisco CA',
 'Hudson County NJ',
 'Suffolk County MA',
 'Philadelphia County PA',
 'Washington DC',
 'Alexandria VA',
 'Baltimore MD',
 'Cook County IL',
 'St. Louis County MO',
 'Milwaukee County WI',
 'Denver County CO',
 'Orange County CA',
 'Pinellas County FL',
 'Ramsey County MN',
 'Wayne County MI',
 'Cuyahoga County OH',
 'Dallas County TX',
 'DeKalb County GA',
 'Salt Lake County UT',
 'Marion County IN',
 'Jefferson County KY',
 'Johnson County KS',
 'King County WA']


locs = ['29.88548725,-96.27736949,100mi',
    '40.7741618671,-73.9697971209,100mi',
    '37.7597150131,-122.693975573,100mi',
    '40.7309049975,-74.0759554422,100mi',
    '42.3486893925,-70.9856096697,100mi',
    '40.0076360466,-75.1339446794,100mi', 
    '38.9047737146,-77.0163026702,100mi',
    '38.8184535854,-77.0862060584,100mi',
    '39.443072852,-76.6163229502,100mi',
    '41.8954290221,-87.6461407931,100mi',
    '38.6406885662,-90.4433975603,100mi',
    '43.0154568883,-87.5805486607,100mi',
    '39.7621646287,-104.875801423,100mi',
    '33.6769134328,-117.776142822,100mi',
    '27.9026762041,-82.7395539464,100mi',
    '45.0170541432,-93.099606274,100mi',
    '42.2847454761,-83.2611328686,100mi',
    '41.6350590585,-81.7001959208,100mi',
    '32.7666300195,-96.7778770663,100mi',
    '33.7715468713,-84.226432416,100mi',
    '40.6673139262,-111.923601307,100mi',
    '39.7817096805,-86.1384716709,100mi',
    '38.1871910002,-85.6591577058,100mi',
    '38.8837685125,-94.8222610425,100mi',
    '47.4909231926,-121.83595173,100mi']
## Just zip locs_labels with locs and you'd have the coordinates mapped to their names

mapped_locations = list(zip(locs_labels, locs))

In [27]:
mapped_locations

[('Texas test', '29.88548725,-96.27736949,100mi'),
 ('New York County NY', '40.7741618671,-73.9697971209,100mi'),
 ('County of San Francisco CA', '37.7597150131,-122.693975573,100mi'),
 ('Hudson County NJ', '40.7309049975,-74.0759554422,100mi'),
 ('Suffolk County MA', '42.3486893925,-70.9856096697,100mi'),
 ('Philadelphia County PA', '40.0076360466,-75.1339446794,100mi'),
 ('Washington DC', '38.9047737146,-77.0163026702,100mi'),
 ('Alexandria VA', '38.8184535854,-77.0862060584,100mi'),
 ('Baltimore MD', '39.443072852,-76.6163229502,100mi'),
 ('Cook County IL', '41.8954290221,-87.6461407931,100mi'),
 ('St. Louis County MO', '38.6406885662,-90.4433975603,100mi'),
 ('Milwaukee County WI', '43.0154568883,-87.5805486607,100mi'),
 ('Denver County CO', '39.7621646287,-104.875801423,100mi'),
 ('Orange County CA', '33.6769134328,-117.776142822,100mi'),
 ('Pinellas County FL', '27.9026762041,-82.7395539464,100mi'),
 ('Ramsey County MN', '45.0170541432,-93.099606274,100mi'),
 ('Wayne County MI', 

In [28]:
len(locs) # ensure that locs and loc_labels lens match; should have through zip funct anyway

25

In [29]:
len(locs_labels)

25

In [ ]:
### TEXAS TEST ####


#tweet_lst=[]
#geoc="40.6546,73.5594,50mi"
#txgeoc = '29.88548725,-96.27736949,100mi'
#migeoc = '25.6112362,-80.55170587,50mi'

    

# Creation of query method using parameters
#try:
    #tweets = tweepy.Cursor(api.search,q='mask free', lang="en", since=date_since, geocode=txgeoc, tweet_mode='extended').items(1000) 
# Pulling information from tweets iterable object
    #tweets_list = [[tweet.created_at, tweet.id, tweet.full_text] for tweet in tweets]
    #tweets_sort = tweets_list.sort()
    #no_dupes = list(k for k,_ in itertools.groupby(tweets_sort))
    #print(tweets_list)
 # Creation of dataframe from tweets list
 # Add or remove columns as you remove tweet information
    #tweets_d = pd.DataFrame(no_dupes)

#except BaseException as e:
    #print('failed on_status,',str(e))


#tweets_f = pd.DataFrame(no_dupes)

## The code that does the actual scraping by location:

In [ ]:
# SCRAPING TWEETS FOR ALL LOCATIONS # 

tweets_list = []
for loc in mapped_locations:
    try:
        tweets = tweepy.Cursor(api.search,q=search_words, lang="en", since=date_since, geocode=loc[1], tweet_mode='extended').items(2) 
        tweets_list.append((loc[0], [[tweet.full_text] for tweet in tweets]))
        print(tweets_list)
        tweets_df = pd.DataFrame(tweets_list)

    except BaseException as e:
        print('failed on_status,',str(e))
        time.sleep(3)
    

In [ ]:
tweets_df

In [ ]:
tweets_df.to_csv('tweet_dataset.csv') # saves the dataframe


################################################################################################

# Tweet df loading and cleaning

In [12]:
import os
import tweepy
import pandas as pd
import time
import itertools
import numpy as np
from itertools import chain
import regex as re
import unicodedata
import nltk

## Loads tweet dataframe, ensures datatypes are appropriate, flattens tweets into one list for ease, labels columns

In [13]:
tweets_df = pd.read_csv('tweet_dataset.csv', converters={'1': eval}, index_col=0)
tweets_df['tweets_list'] = tweets_df['1'].apply(lambda x : np.array(x).flatten())
tweets_df.columns = ['county', 'original_tweet', 'tweets_list']
tweets_df

FileNotFoundError: [Errno 2] No such file or directory: 'tweet_dataset.csv'

In [ ]:
# texas  --- 226 total tweets
#for index, tweet in enumerate(list(tweets_df['tweets_list'].iloc[0])):
#    print(index, tweet)
    

## Now, expands dataset so there is one row per tweet (before, all tweets for a county were in one row)


In [ ]:
t_explode = tweets_df.explode('original_tweet')
t_explode

In [ ]:
# tex = t_explode.loc[t_explode['county'] == 'Texas test']

# print(len(tex)) # confirm that this is 226; it is

***********
# See note:

I tried getting rid of duplicates a few ways; doing pairwise levenshtein distance comparisons between all the tweets took way too long, and creating a mask to then drop duplicate tweets was also complicated. I noticed that most very-similar-basically-duplicate tweets had the same text, but different handles and urls. So I took a slice of part of the text that was very likely to be shared between the similar tweets, created a new column in the dataset where each tweet-row got a corresponding tweet-slice, and then I deleted the rows where there were duplicate tweet-slices. As you can see, this resulted in a *lot* of data loss (3000+ tweets down to 850; see twitter_drop a few cells down), because many of them were duplicate/spam tweets.
***********

In [220]:
# converting to string data type 
t_explode["tweet_str"]= t_explode["original_tweet"].astype(str) 
  
# slicing till 2nd last element 
t_explode["tweet_slice"] = t_explode["tweet_str"].str.slice(50,1000) 
  
# display 
t_explode

,county,original_tweet,tweets_list,tweet_str,tweet_slice
0,Texas test,[@Jim91674418 @fpatrickwelsh @tomselliott @chu...,[@Jim91674418 @fpatrickwelsh @tomselliott @chu...,"[""@Jim91674418 @fpatrickwelsh @tomselliott @ch...",odd Its just not helpful for the discussion or...
0,Texas test,[@zulemashaden But shutting down business are ...,[@Jim91674418 @fpatrickwelsh @tomselliott @chu...,['@zulemashaden But shutting down business are...,y controlling the country like we are not free...
0,Texas test,[@SharpieDj @DianeDenizen @davidgadsby9 @tonyh...,[@Jim91674418 @fpatrickwelsh @tomselliott @chu...,"[""@SharpieDj @DianeDenizen @davidgadsby9 @tony...",ll3 Even when states lift the mask mandates th...
0,Texas test,[What are some of the top compliance &amp; eth...,[@Jim91674418 @fpatrickwelsh @tomselliott @chu...,['What are some of the top compliance &amp; et...,stories from the past week? In the only weekl...
0,Texas test,[@ITHERETWEETER1 @katyekellyeand1 @MySwilly @S...,[@Jim91674418 @fpatrickwelsh @tomselliott @chu...,['@ITHERETWEETER1 @katyekellyeand1 @MySwilly @...,KLORE1 @hellsoasis @DitseaYella @Promethiumban...
...,...,...,...,...,...
12,Denver County CO,[the absolute rage and disgust I feel right no...,[RT @subsix848: @fatherofyr @bergerbell It's c...,['the absolute rage and disgust I feel right n...,my zoology teacher was talking about going bac...
12,Denver County CO,[RT @JBStonestreet: Mississippi third-grader L...,[RT @subsix848: @fatherofyr @bergerbell It's c...,['RT @JBStonestreet: Mississippi third-grader ...,a Booth has to wear a face mask to school. But...
12,Denver County CO,[@HackingDave They need to make like anti-zit ...,[RT @subsix848: @fatherofyr @bergerbell It's c...,['@HackingDave They need to make like anti-zit...,ks lol..but yes hopefully in a few months OTF ...
12,Denver County CO,[RT @JBStonestreet: Mississippi third-grader L...,[RT @subsix848: @fatherofyr @bergerbell It's c...,['RT @JBStonestreet: Mississippi third-grader ...,a Booth has to wear a face mask to school. But...


### Can look through slice list to ensure that the slices look right

In [ ]:
slice_list = t_explode['tweet_slice'].tolist()


#for i, v in enumerate(slice_list):
#    print(i,v)
#slice_list[184]

In [ ]:
twitter_drop = t_explode.drop_duplicates(subset=['tweet_slice'], keep='last')
twitter_drop

## Cleans the tweets

In [ ]:
df_clean = twitter_drop

In [ ]:
# lower case
df_clean['tweet_text'] = df_clean['tweet_str'].str.lower()

# remove url links
df_clean['tweet_text'] = df_clean['tweet_text'].apply(lambda x: re.sub(r'https?:\/\/\S+', '', x))

# remove url/website that didn't use http, is only checking for .com websites 
# so words that are seperated by a . are not removed
df_clean['tweet_text'] = df_clean['tweet_text'].apply(lambda x: re.sub(r"www\.[a-z]?\.?(com)+|[a-z]+\.(com)", '', x))
# remove @mention
df_clean['tweet_text'] = df_clean['tweet_text'].apply(lambda x: re.sub(r'@mention', '', x))
# remove {link}
df_clean['tweet_text'] = df_clean['tweet_text'].apply(lambda x: re.sub(r'{link}', '', x))
# remove &text; html chars
df_clean['tweet_text'] = df_clean['tweet_text'].apply(lambda x: re.sub(r'&[a-z]+;', '', x))
# [video]
df_clean['tweet_text'] = df_clean['tweet_text'].apply(lambda x: re.sub(r"\[video\]", '', x))
# remove all remaining characters that aren't letters, white space, or 
# the following #:)(/\='] that are used in emojis or hashtags
df_clean['tweet_text'] = df_clean['tweet_text'].apply(lambda x: re.sub(r"[^a-z\s\(\-:\)\\\/\];='#]", '', x))
df_clean

## Do a spot check to make sure that the data has been cleaned well:

In [ ]:
# df_clean['tweet_text'].iloc[5]

In [ ]:
# twitter_drop['tweet_str'].iloc[5]

# Save as csv

In [ ]:
df_clean.to_csv('cleaned_tweets.csv')

#########################################################################################################